In [ ]:
#Il y a des passages répétés dans chaque cellule de code, car Jupyter a des conséquences. Ainsi, il faut parfois répéter l'importation ou des initialisations.
#si on prend epochs 100 le resultat devient plus mieux 

In [2]:
import cv2
import os

data_path = 'Training'
categories = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

print(label_dict)
print(categories)
print(labels)

{'female': 0, 'male': 1, 'model-001.model': 2, 'model-002.model': 3, 'model-003.model': 4, 'model-004.model': 5, 'model-006.model': 6}
['female', 'male', 'model-001.model', 'model-002.model', 'model-003.model', 'model-004.model', 'model-006.model']
[0, 1, 2, 3, 4, 5, 6]


In [2]:

#1. `import cv2`: Importe la bibliothèque OpenCV, qui est souvent utilisée pour le traitement d'images en informatique.

#2. `import os`: Importe la bibliothèque os, qui fournit un moyen de travailler avec le système d'exploitation,
#notamment pour effectuer des opérations liées aux fichiers et aux répertoires.

#3. `data_path = 'Training'`: Définit le chemin du répertoire contenant les données d'entraînement.
#Assurez-vous que ce répertoire existe dans le chemin spécifié.

#4. `categories = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]`: Crée une liste `categories` 
#qui contient les noms des sous-répertoires présents dans le répertoire `data_path`. 
#Cela suppose que chaque sous-répertoire représente une catégorie distincte.

#5. `labels = [i for i in range(len(categories))]`: Crée une liste `labels` contenant des nombres de 0 à `len(categories) - 1`. 
#Ces nombres seront utilisés comme étiquettes numériques pour chaque catégorie.

#6. `label_dict = dict(zip(categories, labels))`: Crée un dictionnaire `label_dict` associant chaque catégorie à son étiquette numérique respective.
#La fonction `zip` combine les éléments de `categories` et `labels` par paire, puis `dict` crée un dictionnaire à partir de ces paires.

#7. `print(label_dict)`: Affiche le dictionnaire `label_dict`, montrant la correspondance entre les catégories et les étiquettes.

#8. `print(categories)`: Affiche la liste des catégories.


#9. `print(labels)`: Affiche la liste des étiquettes numériques associées aux catégories.



In [3]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical

def load_images_and_labels(data_path):
    categories = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
    labels = [i for i in range(len(categories))]
    label_dict = dict(zip(categories, labels))

    img_size = 32
    data = []
    target_list = []  # Use a list to append elements
    facedata = "haarcascade_frontalface_default.xml"
    cascade = cv2.CascadeClassifier(facedata)

    for category in categories:
        folder_path = os.path.join(data_path, category)

        img_names = os.listdir(folder_path)

        for img_name in img_names:
            img_path = os.path.join(folder_path, img_name)

            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                faces = cascade.detectMultiScale(img)

                try:
                    for x, y, w, h in faces:
                        sub_face = img[y:y + h, x:x + w]
                        gray = cv2.cvtColor(sub_face, cv2.COLOR_BGR2GRAY)
                        resized = cv2.resize(gray, (img_size, img_size))
                        data.append(resized)
                        target_list.append(label_dict[category])
                except Exception as e:
                    print('Exception:', e)

    # Convert the list to a NumPy array
    data = np.array(data)
    target = np.array(target_list)

    return data, target, label_dict

# Usage
data_path = 'Training'
data, target, label_dict = load_images_and_labels(data_path)

# Use to_categorical from keras.utils
new_target = to_categorical(target)


In [1]:
#def load_images_and_labels(data_path): Définit une fonction load_images_and_labels qui prend le chemin du répertoire de données en entrée et retourne les données, les étiquettes,
#et un dictionnaire associant les catégories à des étiquettes numériques.
##data, target, label_dict = load_images_and_labels(data_path): Appelle la fonction load_images_and_labels avec le chemin du répertoire d'entraînement 
#et récupère les données, les étiquettes, et le dictionnaire de catégories.

#new_target = to_categorical(target): Utilise la fonction to_categorical de Keras pour convertir les étiquettes en représentation one-hot,
#c'est-à-dire une représentation binaire où une seule valeur est "chaude" (1) et les autres sont "froides" (0). Cela est couramment utilisé dans les tâches de classification pour représenter les étiquettes de manière appropriée pour les modèles d'apprentissage automatique.

In [5]:
import numpy as np
import warnings
from keras.utils import to_categorical
img_size = 32
warnings.filterwarnings('ignore')

data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

# Use to_categorical directly from keras.utils
new_target = to_categorical(target)

np.save('./training/data', data)
np.save('./training/target', new_target)


In [3]:
#img_size = 32: Définit la taille d'image à 32 pixels.

#warnings.filterwarnings('ignore'): Ignore les avertissements pendant l'exécution du script.

#data = np.array(data) / 255.0: Normalise les données d'image en divisant chaque pixel par 255.0, ce qui ramène les valeurs de pixel
#dans la plage [0, 1].

#data = np.reshape(data, (data.shape[0], img_size, img_size, 1)): Remodèle les données d'image en un tableau bidimensionnel avec une profondeur de 1. 
#Cela est souvent nécessaire pour les modèles de convolution neurale (CNN) où la profondeur représente le nombre de canaux (par exemple, une seule chaîne pour les images en niveaux de gris).

#target = np.array(target): Convertit les étiquettes en un tableau NumPy.

#new_target = to_categorical(target): Utilise la fonction to_categorical de Keras pour convertir les étiquettes en représentation one-hot.

#np.save('./training/data', data): Sauvegarde les données d'images normalisées dans un fichier Numpy (.npy) sous 
#le nom 'data' dans le répertoire './training/'.



In [6]:
import numpy as np

data=np.load('./training/data.npy')
target=np.load('./training/target.npy')

In [4]:
#import numpy as np: Importe la bibliothèque NumPy.

#data=np.load('./training/data.npy'): Charge les données normalisées à partir du fichier Numpy './training/data.npy' 
#dans la variable data. Ces données ont probablement été prétraitées et sauvegardées à l'aide du script précédent.

#target=np.load('./training/target.npy'): Charge les étiquettes one-hot à partir du fichier Numpy './training/target.npy'
#dans la variable target. Ces étiquettes ont également été sauvegardées lors du prétraitement des données.

In [7]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint


noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=64



model=Sequential()
model.add((Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))
model.add((Conv2D(32, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [5]:

#5. `model=Sequential()`: Initialise un modèle séquentiel.

#6. `model.add(Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu'))`:
#Ajoute une couche de convolution avec 32 filtres de taille (3,3), 
#une fonction d'activation ReLU et une forme d'entrée spécifiée par `input_shape` qui correspond à la taille des données d'images.

#7. `model.add(Conv2D(32, sizeOfFilter1,activation='relu'))`: Ajoute une deuxième couche de convolution avec 32 filtres de taille (3,3) et une fonction
#d'activation ReLU.

#8. `model.add(MaxPooling2D(pool_size=sizeOfPool))`: Ajoute une couche de max pooling avec une fenêtre de pooling de taille (2,2).

#9. `model.add(Conv2D(64, sizeOfFilter2,activation='relu'))`: Ajoute une troisième couche de convolution avec 64 filtres de taille (3,3) 
#et une fonction d'activation ReLU.

#10. `model.add(Conv2D(64, sizeOfFilter2,activation='relu'))`: Ajoute une quatrième couche de convolution avec 64 filtres de taille (3,3) et
#une fonction d'activation ReLU.

#11. `model.add(MaxPooling2D(pool_size=sizeOfPool))`: Ajoute une deuxième couche de max pooling avec une fenêtre de pooling de taille (2,2).

#12. `model.add(Dropout(0.5))`: Ajoute une couche de dropout avec un taux de 0.5 pour régulariser le modèle.

#13. `model.add(Flatten())`: Ajoute une couche de flatten qui convertit les données en un vecteur 1D avant de passer à la couche dense.

#14. `model.add(Dense(noOfNode, activation='relu'))`: Ajoute une couche dense avec 64 nœuds et une fonction d'activation ReLU.

#15. `model.add(Dropout(0.5))`: Ajoute une deuxième couche de dropout avec un taux de 0.5.

#16. `model.add(Dense(2, activation='softmax'))`: Ajoute une dernière couche dense avec 2 nœuds (pour la classification binaire) et 
#une fonction d'activation softmax.

#17. `model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])`: Compile le modèle en spécifiant la fonction de perte (categorical crossentropy,
#adaptée à la classification binaire), l'optimiseur (Adam), et les métriques à suivre (précision ici). Cela prépare le modèle pour l'entraînement.

In [8]:
from sklearn.model_selection import train_test_split    #qui est utilisée pour diviser les données en ensembles d'entraînement et de test.

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)  #Utilise la fonction train_test_split pour diviser les données (data) et les étiquettes (target) en ensembles d'entraînement (train_data, train_target) et ensembles de test (test_data, test_target).

In [10]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
#Initialise un objet de rappel ModelCheckpoint qui surveille la valeur de perte sur l'ensemble de validation (val_loss). 
#Les modèles sont sauvegardés au format HDF5 à des points spécifiques pendant l'entraînement dans le dossier ./training/. L'utilisation de variables telles que {epoch:03d} dans le nom du fichier garantit que le numéro d'époque est inclus dans le nom du modèle sauvegardé.
history=model.fit(train_data,train_target,epochs=2,callbacks=[checkpoint],validation_split=0.2) # Utilise la méthode fit pour entraîner le modèle sur les données d'entraînement

Epoch 1/2
367/367 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5174INFO:tensorflow:Assets written to: ./training\model-001.model\assets


INFO:tensorflow:Assets written to: ./training\model-001.model\assets


367/367 [==============================] - 73s 200ms/step - loss: 0.6928 - accuracy: 0.5174 - val_loss: 0.6928 - val_accuracy: 0.5133
Epoch 2/2
367/367 [==============================] - 80s 219ms/step - loss: 0.6927 - accuracy: 0.5177 - val_loss: 0.6929 - val_accuracy: 0.5133


In [11]:
from keras.models import load_model
import cv2
import numpy as np

In [12]:
model = load_model('C:\\Users\\user\\Desktop\\datasetGender\\Training\\model-006.model')
#Charge le modèle préalablement entraîné à partir du fichier 'model-006.model'. 
#La fonction load_model est généralement fournie par des bibliothèques d'apprentissage automatique, comme Keras.

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')#nitialise un classificateur en cascade Haar pour la détection de visages à partir du fichier XML 'haarcascade_frontalface_default.xml'.
#Ce fichier contient les informations nécessaires pour le détecteur de visage.

cap=cv2.VideoCapture(0)     #Initialise une capture vidéo à partir de la caméra 


labels_dict={0:'Female',1:'Male'}   #Crée un dictionnaire associant les indices de classe (0 et 1) aux étiquettes correspondantes ('Female' et 'Male').
color_dict={0:(0,0,255),1:(0,255,0)} #Crée un dictionnaire associant les indices de classe aux couleurs pour 
#dessiner des cadres de différentes couleurs autour des visages détectés.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Exemple de données d'entraînement
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))

# Créer un modèle simple
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=20))
model.add(Dense(units=1, activation='sigmoid'))

# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Remplir les listes data et labels avec les données d'entraînement
data = x_train.tolist()
labels = y_train.flatten().tolist()

# Diviser les données en ensembles d'entraînement et de test
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convertir les listes en tableaux NumPy
trainX = np.array(trainX)
testX = np.array(testX)
trainY = np.array(trainY)
testY = np.array(testY)

# Entraîner le modèle
epochs = 10
H = model.fit(trainX, trainY, epochs=epochs, batch_size=32, validation_split=0.2)

# Plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss") #Trace la courbe de perte sur l'ensemble d'entraînement (train_loss) en fonction du nombre d'époques.

plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss") #Trace la courbe de perte sur l'ensemble de validation (val_loss) en fonction du nombre d'époques.
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy") #Trace la courbe de précision sur l'ensemble d'entraînement (train_accuracy) en fonction du nombre d'époques.
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy") #Trace la courbe de précision sur l'ensemble de validation (val_accuracy) en fonction du nombre d'époques.
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.show()


In [ ]:
#####################################partie test par webcam
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('C:\\Users\\user\\Desktop\\datasetGender\\Training\\model-006.model')  # Replace with the actual path

# Initialize video capture (modify the parameter for your use case)
cap = cv2.VideoCapture(0)  # Use 0 for default camera, or provide the video file path

if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')  # Replace with the actual path

if face_clsfr.empty():
    print("Error: Could not load face cascade classifier.")
    exit()

labels_dict = {0: 'Female', 1: 'Male'}
color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}

while True:
    # Read a frame
    ret, img = cap.read()

    # Check if the frame is successfully read
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_clsfr.detectMultiScale(gray, 1.3, 3)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+w, x:x+w]
        resized = cv2.resize(face_img, (32, 32))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 32, 32, 1))
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(img, (x, y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(img, (x, y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display the result
    cv2.imshow('Result', img)

    # Check for the 'q' key to exit
    k = cv2.waitKey(1)
    if k == ord("q"):
        break

# Release the video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()






1/1 [==============================] - 0s 71ms/step
